In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import time
import itertools
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
#import seaborn as sns   ---包导入问题 无法解决
import scipy.stats as stats
from scipy.stats import gamma
from scipy.stats import ortho_group

#import cupy as cp
#import cupyx.scipy 

from LFHSIC.fhsic_naive import IndpTest_naive_rff
from LFHSIC.lfhsic_g import IndpTest_LFGaussian
from LFHSIC.lfhsic_m import IndpTest_LFMahalanobis

# 原代码（需GPU）- 改为强制使用CPU
#device = torch.device('cuda:0')
device = torch.device('cpu')


In [1]:
import sys
print(sys.path)  # 查看 Python 路径
!{sys.executable} -m pip show seaborn  # 确认 Seaborn 安装位置

['D:\\python\\python313.zip', 'D:\\python\\DLLs', 'D:\\python\\Lib', 'D:\\python', 'd:\\码仓\\001_nips_lfhsic_code\\.venv', '', 'd:\\码仓\\001_nips_lfhsic_code\\.venv\\Lib\\site-packages', 'd:\\码仓\\001_nips_lfhsic_code\\.venv\\Lib\\site-packages\\win32', 'd:\\码仓\\001_nips_lfhsic_code\\.venv\\Lib\\site-packages\\win32\\lib', 'd:\\码仓\\001_nips_lfhsic_code\\.venv\\Lib\\site-packages\\Pythonwin']


In [ ]:
# n = 10000, d = 16, D=300
def generate_ISA(n,d,sigma_normal,alpha):
    
    x = np.concatenate((np.random.normal(-1, sigma_normal, n//2), np.random.normal(1, sigma_normal, n//2)))
    y = np.concatenate((np.random.normal(-1, sigma_normal, n//2), np.random.normal(1, sigma_normal, n//2)))
    p = np.random.permutation(n)
    y_p = y[p]

    D = np.zeros([2,n])
    D[0,:] = x
    D[1,:] = y_p

    theta = np.pi/4*alpha
    c, s = np.cos(theta), np.sin(theta)
    R = np.array(((c, -s), (s, c)))

    D_R = R@D
    X_mix = D_R[0,:].reshape(-1,1)
    Y_mix = D_R[1,:].reshape(-1,1)

    X_z = np.random.randn(n,d-1)
    Y_z = np.random.randn(n,d-1)

    X_con = np.concatenate((X_mix,X_z), axis=1)
    Y_con = np.concatenate((Y_mix,Y_z), axis=1)

    m_x = ortho_group.rvs(dim=d)
    m_y = ortho_group.rvs(dim=d)

    X = (m_x@X_con.T).T
    Y = (m_y@Y_con.T).T
    
    return X,Y

# w = 5
def Sinusoid(x, y, w):
    return 1 + np.sin(w*x)*np.sin(w*y)

def Sinusoid_Generator(n,w):
    i = 0
    output = np.zeros([n,2])
    while i < n:
        U = np.random.rand(1)
        V = np.random.rand(2)
        x0 = -np.pi + V[0]*2*np.pi
        x1 = -np.pi + V[1]*2*np.pi
        if U < 1/2 * Sinusoid(x0,x1,w):
            output[i, 0] = x0
            output[i, 1] = x1
            i = i + 1
    return output[:,0], output[:,1]

# n = 3000
# d = 4
def sinedependence(n,d):
    mean = np.zeros(d)
    cov = np.eye(d)
    X = np.random.multivariate_normal(mean, cov, n)
    Z = np.random.randn(n)
    Y = 20*np.sin(4*np.pi*(X[:,0]**2 + X[:,1]**2))+Z 
    return X,Y

# n = 4000
# d = 5
def GSign(n,d):
    mean = np.zeros(d)
    cov = np.eye(d)
    X = np.random.multivariate_normal(mean, cov, n)
    sign_X = np.sign(X)
    Z = np.random.randn(n)
    Y = np.abs(Z)*np.prod(sign_X,1)
    return X,Y

In [ ]:
n = 3000
d = 4
D = 100
test_num = 50

result_test_correct = np.zeros([2,test_num])

for it in range(test_num):
    X, Y = sinedependence(n, d)
    Y = Y.reshape(-1,1)
    
    rand_index = np.random.permutation(n)
    X = X[rand_index]
    Y = Y[rand_index]
    X_tensor, Y_tensor = torch.tensor(X), torch.tensor(Y)

    if len(X_tensor.size())==1:
        X_tensor = X_tensor.reshape(-1,1)
    if len(Y_tensor.size())==1:
        Y_tensor = Y_tensor.reshape(-1,1)

    #test#

    lfhsic_g = IndpTest_LFGaussian(X_tensor, Y_tensor, device, alpha=0.05, null_gamma = True, split_ratio = 0.5)
    results_all1 = lfhsic_g.perform_test(rff_num = D, lr = 0.05, if_grid_search = True, debug = -1)
    result_test_correct[0, it] = float(results_all1['h0_rejected'])

    lfhsic_m = IndpTest_LFMahalanobis(X_tensor, Y_tensor, device, alpha=0.05, null_gamma = True, split_ratio = 0.5)
    results_all2 = lfhsic_m.perform_test(rff_num = D, lr = 0.05, if_grid_search = True, debug = -1)
    result_test_correct[1, it] = float(results_all2['h0_rejected'])

    print(it, np.sum(result_test_correct,1)/(it+1))

In [ ]:
n = 2000
w = 5
D = 100
test_num = 50

result_test_correct = np.zeros([2,test_num])

for it in range(test_num):
    output = Sinusoid_Generator(n,w)
    X = output[0].reshape(-1,1)
    Y = output[1].reshape(-1,1)
    
    rand_index = np.random.permutation(n)
    X = X[rand_index]
    Y = Y[rand_index]
    X_tensor, Y_tensor = torch.tensor(X), torch.tensor(Y)

    if len(X_tensor.size())==1:
        X_tensor = X_tensor.reshape(-1,1)
    if len(Y_tensor.size())==1:
        Y_tensor = Y_tensor.reshape(-1,1)

    #test#

    lfhsic_g = IndpTest_LFGaussian(X_tensor, Y_tensor, device, alpha=0.05, null_gamma = True, split_ratio = 0.5)
    results_all1 = lfhsic_g.perform_test(rff_num = D, lr = 0.05, if_grid_search = True, debug = -1)
    result_test_correct[0, it] = float(results_all1['h0_rejected'])

    lfhsic_m = IndpTest_LFMahalanobis(X_tensor, Y_tensor, device, alpha=0.05, null_gamma = True, split_ratio = 0.5)
    results_all2 = lfhsic_m.perform_test(rff_num = D, lr = 0.05, if_grid_search = True, debug = -1)
    result_test_correct[1, it] = float(results_all2['h0_rejected'])

    print(it, np.sum(result_test_correct,1)/(it+1))

In [ ]:
n = 4000
d = 5
D = 100
test_num = 50

result_test_correct = np.zeros([2,test_num])

for it in range(test_num):
    X, Y = GSign(n,d)
    Y = Y.reshape(-1,1)
    
    rand_index = np.random.permutation(n)
    X = X[rand_index]
    Y = Y[rand_index]
    X_tensor, Y_tensor = torch.tensor(X), torch.tensor(Y)

    if len(X_tensor.size())==1:
        X_tensor = X_tensor.reshape(-1,1)
    if len(Y_tensor.size())==1:
        Y_tensor = Y_tensor.reshape(-1,1)

    #test#

    lfhsic_g = IndpTest_LFGaussian(X_tensor, Y_tensor, device, alpha=0.05, null_gamma = True, split_ratio = 0.5)
    results_all1 = lfhsic_g.perform_test(rff_num = D, lr = 0.05, if_grid_search = True, debug = -1)
    result_test_correct[0, it] = float(results_all1['h0_rejected'])

    lfhsic_m = IndpTest_LFMahalanobis(X_tensor, Y_tensor, device, alpha=0.05, null_gamma = True, split_ratio = 0.5)
    results_all2 = lfhsic_m.perform_test(rff_num = D, lr = 0.05, if_grid_search = True, debug = -1)
    result_test_correct[1, it] = float(results_all2['h0_rejected'])

    print(it, np.sum(result_test_correct,1)/(it+1))

In [ ]:
# ica experiments #
D = 500
test_num = 50
result_all_correct = []

# alpha_set = np.linspace(0.0, 1.0, num=10)
alpha_set = [1.0]

confident_level = 0.05

for alpha in alpha_set:
    result_test_correct = np.zeros([2,test_num])
    
    for it in range(test_num):
        n = 10000
        d = 16
        sigma_normal = 0.1
        
        X, Y = generate_ISA(n,d,sigma_normal,alpha)
        rand_index = np.random.permutation(n)
        X = X[rand_index]
        Y = Y[rand_index]
        
        X_tensor, Y_tensor = torch.tensor(X), torch.tensor(Y)
        
        #test#

        lfhsic_g = IndpTest_LFGaussian(X_tensor, Y_tensor, device, alpha=0.05, null_gamma = True, split_ratio = 0.5)
        results_all1 = lfhsic_g.perform_test(rff_num = D, lr = 0.05, if_grid_search = False, debug = -1)
        result_test_correct[0, it] = float(results_all1['h0_rejected'])

        lfhsic_m = IndpTest_LFMahalanobis(X_tensor, Y_tensor, device, alpha=0.05, null_gamma = True, split_ratio = 0.5)
        results_all2 = lfhsic_m.perform_test(rff_num = D, lr = 0.05, if_grid_search = False, debug = -1)
        result_test_correct[1, it] = float(results_all2['h0_rejected'])

        print(alpha, it, np.sum(result_test_correct,1)/(it+1))
    
    result_all_correct.append(result_test_correct)